# Loading Trained Model

In [19]:
import torch
from torch import nn
from torch.utils.data import DataLoader

from src.Mymodel import MyResNet34
from src.Mymodel import MyResNet_CIFAR
from src.Mydataloader import LoadDataset
from src.Mytraining import DoTraining

In [20]:
"""Dataset selection"""
DATASET = "CIFAR10"
# DATASET = "CIFAR100"
# DATASET = "ImageNet2012"

"""Model selection for CIFAR"""
NUM_LAYERS_LEVEL = 5

"""Dataset parameters"""
BATCH = 128
SHUFFLE = True
NUMOFWORKERS = 8
PIN_MEMORY = True
SPLIT_RATIO = 0

"""optimizer parameters"""
OPTIMIZER = "SGD"
# OPTIMIZER = "Adam"
# OPTIMIZER = "Adam_decay"


file_path = ""
if DATASET == "ImageNet2012":
    file_path = f"MyResNet34_{BATCH}_{OPTIMIZER}"
    _model_name = f"MyResNet34_{DATASET}_{BATCH}_{OPTIMIZER}"

    # casenum = 1
    # if casenum != 0:
    #     _model_name += f"_case{casenum}"
else:
    file_path = f"MyResNet{NUM_LAYERS_LEVEL*6+2}_{BATCH}_{OPTIMIZER}"
    _model_name = f"MyResNet{NUM_LAYERS_LEVEL*6+2}_{DATASET}_{BATCH}_{OPTIMIZER}"

if SPLIT_RATIO != 0:
    _model_name += f"_{int(SPLIT_RATIO*100)}"
    file_path += f"_{int(SPLIT_RATIO*100)}"

In [21]:
tmp = LoadDataset(root="data", seceted_dataset=DATASET, split_ratio=SPLIT_RATIO)
_, valid_data, test_data, COUNT_OF_CLASSES = tmp.Unpack()

if valid_data is not None:
    valid_dataloader = DataLoader(
        valid_data,
        batch_size=BATCH,
        shuffle=SHUFFLE,
        num_workers=NUMOFWORKERS,
        pin_memory=PIN_MEMORY,
        # pin_memory_device="cuda",
        persistent_workers=True,
    )
    print("valid.transforms =", valid_data.transform, valid_dataloader.batch_size)
else:
    valid_dataloader = None

if test_data is not None:
    test_dataloader = DataLoader(
        test_data,
        batch_size=BATCH,
        shuffle=SHUFFLE,
        num_workers=NUMOFWORKERS,
        pin_memory=PIN_MEMORY,
        # pin_memory_device="cuda",
        persistent_workers=True,
    )
    print("test.transforms =", test_data.transform, test_dataloader.batch_size)
else:
    test_dataloader = None
    
"""Model selection"""   
if valid_dataloader is None:
    eval_dataloader = test_dataloader
elif test_dataloader is None:
    eval_dataloader = valid_dataloader
else:
    raise ValueError("valid_dataloader and test_dataloader cannot be None at the same time")

-----------------------------------------------------------------------
Dataset :  CIFAR10
- Length of Train Set :  50000
- Length of Test Set :  10000
- Count of Classes :  10
-----------------------------------------------------------------------
test.transforms = ToTensor() 128


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if DATASET == "ImageNet2012":
    model = MyResNet34(num_classes=COUNT_OF_CLASSES, Downsample_option="B").to(device)

else:
    model = MyResNet_CIFAR(
        num_classes=COUNT_OF_CLASSES, num_layer_factor=NUM_LAYERS_LEVEL
    ).to(device)



model.load_state_dict(torch.load(f"models/{_model_name}/{file_path}.pth"))

<All keys matched successfully>

In [23]:
testingclass = DoTraining(
    model=model, criterion=nn.CrossEntropyLoss(), optimizer=None, device=device
)
test_loss, test_acc = testingclass.Forward_eval(eval_dataloader)

In [24]:
print("test_loss:", test_loss)

test_loss: 0.2317420024471947


In [25]:
print(f"test_acc: {test_acc*100:.2f}%")


test_acc: 92.63%


In [26]:
print(f"test_error: {100 - test_acc*100:.2f}%")

test_error: 7.37%


In [27]:
file_path

'MyResNet32_128_SGD'